# Combine national and NERC data
Take the various file (EIA facilities, EIA state-level totals, EPA emissions, etc) and combine them to determine total generation, CO₂ emissions, etc nationally and at NERC regions.

## Instructions
Make sure the `file_date` parameter below is set to whatever value you would like appended to file names.

In **Calculate the extra gen/fuel consumption at the state levels** add years after 2017 in the `for` loop if needed.

In **Fraction of estimated gen/fuels in each NERC region** update the years that plant ids for annual plants are fetched from EIA-923 files.

The entire notebook can be run at once using *Run All Cells*

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import os
from os.path import join
import sys
import json


idx = pd.IndexSlice
cwd = os.getcwd()
# data_path = join(cwd, '..', 'Data storage')

### Date string for filenames
This will be inserted into all filenames (reading and writing)

In [ ]:
file_date = '2018-06-15'

In [ ]:
%load_ext watermark
%watermark -iv -v

In [ ]:
# Load the "autoreload" extension
%load_ext autoreload

# always reload modules marked with "%aimport"
%autoreload 1

In [ ]:
# add the 'src' directory as one where we can import modules
src_dir = join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

In [86]:
from src.analysis.index import (
    facility_emission_gen,
    group_facility_data,
    g2lb,
    change_since_2005,
    generation_index,
    facility_co2,
    adjust_epa_emissions,
    group_fuel_cats,
    extra_emissions_gen,
    reduce_emission_factors
)
from src.util import add_quarter, add_facility_location, rename_cols
from src.params import (
    DATA_DATE,
    DATA_PATHS,
    QUARTER_YEAR,
    STATE_FACILITY_FUELS,
    CUSTOM_FUELS,
    FINAL_DATA_YEAR,
    STATES,
    NERCS
)

prev_year = FINAL_DATA_YEAR - 1

ImportError: cannot import name 'NERCS'

> <ipython-input-86-aa585f854673>(14)<module>()
     12 )
     13 from src.util import add_quarter, add_facility_location, rename_cols
---> 14 from src.params import (
     15     DATA_DATE,
     16     DATA_PATHS,



In [ ]:
# %aimport src.data.make_data
# from src.data.make_data import states_in_nerc
%aimport src.analysis.index
from Analysis.index import facility_emission_gen, group_facility_data, add_quarter
%aimport Analysis.index
from Analysis.index import g2lb, change_since_2005, generation_index
%aimport Analysis.index
from Analysis.index import facility_co2, adjust_epa_emissions, group_fuel_cats
%aimport src.util
from src.util import rename_cols, add_facility_location

## Load data

In [ ]:
# cwd = os.getcwd()
# path = join(data_path, 'Derived data',
#             'Facility gen fuels and CO2 {}.csv'.format(file_date))


eia_fac = pd.read_csv(path)

In [ ]:
rename_cols(eia_fac)

In [ ]:
path = join(data_path, 'Derived data',
            'Monthly EPA emissions {}.csv'.format(file_date))
epa = pd.read_csv(path)

In [4]:
FACILITY_PATH = (
    DATA_PATHS['eia_compiled']
    / 'facility_gen_fuel_data_{}.parquet'.format(DATA_DATE)
)
eia_fac = pd.read_parquet(FACILITY_PATH)

epa_path = (
    DATA_PATHS['epa_emissions']
    / 'epa_emissions_{}.parquet'.format(DATA_DATE)
)
epa = pd.read_parquet(epa_path)

## Adjusted facility emissions and generation
I have both the state and custom fuel categories here, but am only using the state fuel categories for now.

In [ ]:
fuel_cat_folder = join(data_path, 'Fuel categories')
state_cats_path = join(fuel_cat_folder, 'State_facility.json')

with open(state_cats_path, 'r') as f:
    state_fuel_cat = json.load(f)
    
custom_cats_path = join(fuel_cat_folder, 'Custom_results.json')
with open(custom_cats_path, 'r') as f:
    custom_fuel_cat = json.load(f)

In [81]:
co2, gen_fuels_state = facility_emission_gen(eia_facility=eia_fac, epa=epa,
                                              state_fuel_cat=STATE_FACILITY_FUELS,
                                              custom_fuel_cat=CUSTOM_FUELS,
                                              export_state_cats=True)

Renaming columns
Grouping facilities
Adjusting EPA emissions
Caculating CO2
Gen/fuels to state categories


## Extra gen/fuels from non-reporting

In [ ]:
%aimport Analysis.index
from Analysis.index import extra_emissions_gen

Total EIA generation/fuel consumption and emission factors

In [ ]:
cwd = os.getcwd()
path = join(data_path, 'Derived data',
            'EIA country-wide gen fuel CO2 {}.csv'.format(file_date))
eia_total = pd.read_csv(path)

path = join(data_path,
            'Final emission factors.csv')
ef = pd.read_csv(path, index_col=0)

In [6]:
EF_PATH = DATA_PATHS['inputs'] / 'Final emission factors.csv'
EF = pd.read_csv(EF_PATH, index_col=0)

TOTAL_EIA_PATH = (
    DATA_PATHS['eia_compiled']
    / f'national_gen_fuel_data_{DATA_DATE}.parquet')
eia_total = pd.read_parquet(TOTAL_EIA_PATH)
eia_total.reset_index(inplace=True)

### Calculate CO₂, generation, and fuel consumption that is not captured by facilities

In [82]:
extra_co2, extra_gen_fuel = extra_emissions_gen(gen_fuels_state, eia_total, EF)

In [83]:
extra_gen_fuel.sample(5)

total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)
type year month                                                         
NUC  2001 6           -7.103733e+08             0.000      -7.103733e+08
OTH  2014 9           -1.994694e+07          2817.196      -1.694408e+07
GEO  2004 7           -1.178739e+07        101379.390      -1.178739e+07
     2006 4           -1.119944e+07            -0.002      -1.119944e+07
     2009 10          -1.191773e+07             0.004      -1.191773e+07

## Total CO₂ (national)
Combine adjusted CO₂ at facilities and CO₂ from fuel consumption that is not captured by facility data.

In [8]:
facility_co2 = co2.groupby(['year', 'month']).sum()

In [9]:
national_co2 = (facility_co2.loc[:, 'final co2 (kg)']
                + extra_co2.loc[:, 'elec fuel co2 (kg)']
                           .groupby(['year', 'month']).sum())
national_co2.name = 'final co2 (kg)'

## National Index and gen by fuels

### Total national generation (all fuels)
Add generation/fuel consumption reported by facilities (with state-level fuel codes) and extra generation/fuel consumption (which also uses state-level fuel codes).

In [ ]:
gen_fuels_state.groupby(['type', 'year', 'month']).sum().head()

In [10]:
national_gen = (gen_fuels_state
                .groupby(['type', 'year', 'month'])['generation (mwh)'].sum()
                .add(extra_gen_fuel['generation (mwh)'], fill_value=0))

#### Regroup generation from state codes to my custom fuel codes

In [11]:
national_gen = group_fuel_cats(national_gen.reset_index(), CUSTOM_FUELS,
                               'type', 'fuel category').set_index(['fuel category', 'year', 'month'])

In [12]:
total_gen = national_gen.groupby(['year', 'month']).sum()

### National Index

In [13]:
national_index = total_gen.copy()
national_index['final co2 (kg)'] = national_co2
national_index['index (g/kwh)'] = (national_index['final co2 (kg)']
                                   / national_index['generation (mwh)'])
national_index.reset_index(inplace=True)
add_quarter(national_index)
g2lb(national_index)
change_since_2005(national_index)

In [ ]:
path = join(data_path, 'National data',
            'National index {}.csv'.format(file_date))
national_index.to_csv(path, index=False)

In [14]:
monthly_index = national_index.copy()


quarterly_index = monthly_index.groupby(['year', 'quarter'])['generation (mwh)', 'final co2 (kg)'].sum()
quarterly_index.reset_index(inplace=True)
quarterly_index['index (g/kwh)'] = quarterly_index.loc[:, 'final co2 (kg)'] / quarterly_index.loc[:, 'generation (mwh)']
quarterly_index['year_quarter'] = quarterly_index['year'].astype(str) + ' Q' + quarterly_index['quarter'].astype(str)
change_since_2005(quarterly_index)
g2lb(quarterly_index)

annual_index = quarterly_index.groupby('year')['generation (mwh)', 'final co2 (kg)'].sum()
annual_index.reset_index(inplace=True)
annual_index['index (g/kwh)'] = annual_index.loc[:, 'final co2 (kg)'] / annual_index.loc[:, 'generation (mwh)']

change_since_2005(annual_index)
g2lb(annual_index)

In [93]:
monthly_index.tail()

,year,month,generation (mwh),final co2 (kg),index (g/kwh),datetime,quarter,index (lb/mwh),change since 2005
211,2018,8,4.107660e+08,1.869340e+11,455.086314,2018-08-01,3,1003.283289,-0.240417
212,2018,9,3.592436e+08,1.597507e+11,444.686247,2018-09-01,3,980.355300,-0.257775
213,2018,10,3.277448e+08,1.404711e+11,428.598987,2018-10-01,4,944.889326,-0.284627
214,2018,11,3.237941e+08,1.376966e+11,425.259679,2018-11-01,4,937.527487,-0.290200
215,2018,12,3.388554e+08,1.452962e+11,428.785259,2018-12-01,4,945.299982,-0.284316


In [ ]:
# path = join(data_path, 'Website results',
#             'Monthly index {}.csv'.format(export_quarter))
path = DATA_PATHS['results'] / f'Monthly index {QUARTER_YEAR}.csv'
monthly_index.to_csv(path, index=False)

In [ ]:
# path = join(data_path, 'Website results',
#             'Monthly index {}.csv'.format(export_quarter))
path = DATA_PATHS['results'] / f'Quarterly index {QUARTER_YEAR}.csv'
quarterly_index.to_csv(path, index=False)

In [ ]:
# path = join(data_path, 'Website results',
#             'Monthly index {}.csv'.format(export_quarter))
path = DATA_PATHS['results'] / f'Annual index {QUARTER_YEAR}.csv'
annual_index.to_csv(path, index=False)

### CO₂ intensity of each fuel category, plus files for website

In [87]:
category_ef = reduce_emission_factors(EF)

In [90]:
category_ef

{'NG': 53.07, 'PEL': 75.975, 'PC': 102.1, 'COW': 95.25, 'OOG': 59.0}

In [92]:
pd.Series(category_ef, name='fuel category') * national_gen.sample(5)

,,,COW,NG,OOG,PC,PEL,generation (mwh)
fuel category,year,month,,,,,,
Hydro,2009,4,NaN,NaN,NaN,NaN,NaN,NaN
Nuclear,2010,12,NaN,NaN,NaN,NaN,NaN,NaN
Other,2011,7,NaN,NaN,NaN,NaN,NaN,NaN
Nuclear,2005,7,NaN,NaN,NaN,NaN,NaN,NaN
Natural Gas,2001,4,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
national_gen.sample(5)

generation (mwh)
fuel category year month                  
Hydro         2014 10          17159212.27
Wind          2009 11           6875183.04
              2001 9             490229.00
Nuclear       2010 6           68301321.56
Hydro         2015 2           22286076.01

In [ ]:
national_gen['elec fuel fossil co2 (kg)'] = 0
for fuel in category_ef.keys():
    national_gen.loc[idx[fuel, :, :], 'elec fuel fossil co2 (kg)'] = (
        national_gen.loc[idx[fuel, :, :], 'elec fuel (mmbtu)'] 
        * category_ef[fuel]
    )

national_gen['elec fuel fossil co2 (kg)'] = (national_gen['elec fuel fossil co2 (kg)'] 
                                             + extra_co2['elec fuel co2 (kg)'])

### Percent generation by custom fuel type

In [ ]:
df_list = []
for fuel in national_gen.index.get_level_values('fuel category').unique():
    percent_gen = national_gen.loc[fuel].divide(total_gen, fill_value=0)
    percent_gen['fuel category'] = fuel
    percent_gen.set_index('fuel category', inplace=True, append=True)
    df_list.append(percent_gen)
percent_gen = pd.concat(df_list)

In [ ]:
path = join(data_path, 'National data',
            'National generation {}.csv'.format(file_date))
national_gen.to_csv(path)

path = join(data_path, 'National data',
            'National percent gen {}.csv'.format(file_date))
percent_gen.to_csv(path)

## Facility state and lat/lon file generation
~~Create a .csv with lat/lon and state code for each facility. Need to manually add the NERC region label.~~ ~~I've done this with a spatial join between the lat/lon and NERC shapefiles in QGIS. It should be possible to do the spatial join in [GeoPandas](http://geopandas.org/).  Could also add custom region labels (e.g. eGRID subregions, ISO/RTO boundaries, etc).~~

This isn't possible with a spatial join because NERC regions aren't strictly based on geography. Instead, I've taken the NERC codes from EIA-860 and assigned unknown plants (mostly those that retired before 2012 when modern NERCs were mostly defined, and those that were assigned a plant id in 2017) NERC labels using a k-nearest neighbors algorithm and lat/lon information.

See the `Assign NERC region labels` notebook for the creation of this file.

## Fraction of estimated gen/fuels in each NERC region

**NOTE** EIA changed the reporting requirements for wind and solar facilities in 2017. I'm now going to use the list of annual facilities from 2017 rather than from 2015, but still use the reported generation by those facilities in 2015.

In [ ]:
%aimport Data.make_data
from Data.make_data import get_annual_plants

In [15]:
from src.data.make_data import get_annual_plants

In [17]:
%pdb

Automatic pdb calling has been turned ON


In [25]:
annual_ids_2015 = get_annual_plants(2015)

In [29]:
annual_ids_2017 = get_annual_plants('2017')

In [32]:
prev_annual_ids = get_annual_plants(prev_year)

In [ ]:
annual_ids_2017

Combine the lists of annual plants in 2015 and 2017. This lets us catch facilities that have gone from monthly to annual since 2015, but it also includes plants that were annual in 2015 an may have retired.

There is the possibility of an error in allocation to NERC regions for 2016 state-level generation when using the facilities that changed to annual in 2017. But since the state-level generation is so much smaller in 2016 I don't think this is much of an issue.

In [ ]:
annual_ids = set(annual_ids_2015.tolist() + annual_ids_2017.tolist())

In [ ]:
len(annual_ids)

### Add NERC region labels

In [ ]:
%aimport Analysis.state2nerc
from Analysis.state2nerc import fraction_state2nerc, add_region

In [27]:
from src.analysis.state2nerc import fraction_state2nerc, add_region

In [28]:
# cwd = os.getcwd()
# path = join(data_path, 'Facility labels',
#             'Facility locations_RF.csv')
location_path = DATA_PATHS['transformed_data'] / 'Facility locations_RF.csv'
location_labels = pd.read_csv(location_path)

In [ ]:
nerc_state_path = join(data_path, 'Derived data',
                  'NERC_states.json')

with open(nerc_state_path, 'r') as f:
    nerc_states = json.load(f)

Added the filter that year must be 2015 - was getting all 2015 annual plants, but for all years!

In [ ]:
eia_2015_annual = eia_fac.loc[(eia_fac['plant id'].isin(annual_ids)) & 
                              (eia_fac['year'] == 2015)].copy()

# Group to state-level fuel categories
eia_2015_annual = group_fuel_cats(eia_2015_annual, state_fuel_cat)

In [ ]:
eia_2015_annual_nerc = add_facility_location(eia_2015_annual, location_labels, 
                                        labels=['state', 'nerc', 'year'])

In [ ]:
eia_2015_annual_nerc.head()

In [33]:
eia_prev_annual = eia_fac.loc[(eia_fac['plant id'].isin(prev_annual_ids)) & 
                              (eia_fac['year'] == prev_year)].copy()

# Group to state-level fuel categories
eia_prev_annual = group_fuel_cats(eia_prev_annual, STATE_FACILITY_FUELS)


In [36]:
eia_prev_annual = eia_fac.loc[(eia_fac['plant id'].isin(prev_annual_ids)) & 
                              (eia_fac['year'] == prev_year)].copy()

# Group to state-level fuel categories
eia_prev_annual_nerc = (
    eia_prev_annual.pipe(group_fuel_cats, fuel_cats=STATE_FACILITY_FUELS)
                   .pipe(add_facility_location,
                         label_df=location_labels,
                         labels=['state', 'nerc', 'year'])
)


In [37]:
eia_prev_annual.head()

,f,fuel,month,plant id,total fuel (mmbtu),year,generation (mwh),elec fuel (mmbtu),last_updated,lat,lon,prime mover,state,datetime,quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg),type
0,M,OBL,12,10120,1260.0,2017,0.0,38.0,2019-03-01T00:40:06-05:00,31.164772,-81.478724,ALL,GA,2017-12-01,4,0.0,0.0,105814.80,3191.24,WAS
1,M,OBL,11,10120,1167.0,2017,0.0,36.0,2019-03-01T00:40:06-05:00,31.164772,-81.478724,ALL,GA,2017-11-01,4,0.0,0.0,98004.66,3023.28,WAS
2,M,OBL,10,10120,1414.0,2017,0.0,43.0,2019-03-01T00:40:06-05:00,31.164772,-81.478724,ALL,GA,2017-10-01,4,0.0,0.0,118747.72,3611.14,WAS
3,M,OBL,9,10120,1425.0,2017,0.0,44.0,2019-03-01T00:40:06-05:00,31.164772,-81.478724,ALL,GA,2017-09-01,3,0.0,0.0,119671.50,3695.12,WAS
4,M,OBL,8,10120,1770.0,2017,0.0,54.0,2019-03-01T00:40:06-05:00,31.164772,-81.478724,ALL,GA,2017-08-01,3,0.0,0.0,148644.60,4534.92,WAS


In [38]:
eia_prev_annual_nerc.head()

,type,year,month,plant id,total fuel (mmbtu),generation (mwh),elec fuel (mmbtu),lat,lon,quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg),state,nerc
0,COW,2017,1,59,485196.0,42216.979,485196.0,40.854765,-98.348222,1,47161051.2,47161051.2,47161051.2,47161051.2,NE,MRO
1,COW,2017,1,79,302267.0,15642.000,198135.0,64.847743,-147.735063,1,29380352.4,19258722.0,29380352.4,19258722.0,AK,ASCC
2,COW,2017,1,508,0.0,0.000,0.0,38.033327,-102.537915,1,0.0,0.0,0.0,0.0,CO,WECC
3,COW,2017,1,728,0.0,0.000,0.0,33.462200,-84.898600,1,0.0,0.0,0.0,0.0,GA,SERC
4,COW,2017,1,753,18385.0,721.087,18385.0,31.844708,-83.940734,1,1715320.5,1715320.5,1715320.5,1715320.5,GA,SERC


In [ ]:
eia_prev_annual.pipe()

In [ ]:
eia_2015_annual_nerc = add_facility_location(eia_2015_annual, location_labels, 
                                        labels=['state', 'nerc', 'year'])

This is 2015 data on annual reporting facilities (from both 2015 and 2017)

In [ ]:
# Get a list of all state abbreviations

all_states = []
for value in nerc_states.values():
    all_states.extend(value)
all_states = set(all_states)

In [43]:
df_list = []

for state in STATES:
    try:
        df_list.append(fraction_state2nerc(eia_prev_annual_nerc,
                       state, region_col='nerc', fuel_col='type'))
    except:
        print(state)
        pass

In [44]:
nerc_fraction = pd.concat(df_list)
nerc_fraction.set_index(['state', 'nerc', 'type'], inplace=True)
nerc_fraction.sort_index(inplace=True)

With the values below I can allocate extra state-level generation and fuel use to each of the NERC regions!

Making sure that no values are greater than 1 (within tolerance)

In [45]:
(nerc_fraction.groupby(['state', 'type']).sum() > 1.0001).any()

% generation    False
% total fuel    False
% elec fuel     False
dtype: bool

## Allocate extra gen from the state-level to regions

I still need to generate state-level total generation and fuel use!

In [ ]:
idx = pd.IndexSlice

In [46]:
# a dictionary to match column names
nerc_frac_match = {'% generation': 'generation (mwh)',
                   '% total fuel': 'total fuel (mmbtu)',
                   '% elec fuel': 'elec fuel (mmbtu)'}

### Load state-level total gen/fuel consumption

In [48]:
# path = join(data_path, 'Derived data',
#             'EIA state-level gen fuel CO2 {}.csv'.format(file_date))
STATE_PATH = (
    DATA_PATHS['eia_compiled']
    / f'state_gen_fuel_data_{DATA_DATE}.parquet'
)

state_total = pd.read_parquet(STATE_PATH)

In [49]:
state_total.columns

Index(['generation (MWh)', 'total fuel (mmbtu)', 'elec fuel (mmbtu)',
       'all fuel CO2 (kg)', 'elec fuel CO2 (kg)', 'datetime', 'quarter'],
      dtype='object')

In [58]:
rename_cols(state_total)
# state_total['state'] = state_total['geography'].str[-2:]

Simplify the dataframe

In [59]:
state_total.sample(5)

,,,,generation (mwh),total fuel (mmbtu),elec fuel (mmbtu),all fuel co2 (kg),elec fuel co2 (kg),datetime,quarter
type,year,month,state,,,,,,,
SPV,2017,2,NY,12428.09,NaN,NaN,NaN,NaN,2017-02-01,1
OTH,2002,4,TN,662.32,NaN,NaN,NaN,NaN,2002-04-01,2
HPS,2005,12,MI,-87093.00,NaN,NaN,NaN,NaN,2005-12-01,4
WWW,2014,3,LA,200558.68,NaN,NaN,NaN,NaN,2014-03-01,1
NUC,2008,4,FL,2376495.00,NaN,NaN,NaN,NaN,2008-04-01,2


In [60]:
nerc_frac_match

{'% generation': 'generation (mwh)',
 '% total fuel': 'total fuel (mmbtu)',
 '% elec fuel': 'elec fuel (mmbtu)'}

In [61]:
cols = list(nerc_frac_match.values())
state_total = state_total.groupby(['state', 'year', 'month', 'type'])[cols].sum()

In [62]:
state_total.sample(5)

,,,,generation (mwh),total fuel (mmbtu),elec fuel (mmbtu)
state,year,month,type,,,
KY,2009,1,NG,115770.67,1599950.0,1315980.0
NY,2016,4,NUC,2510168.00,0.0,0.0
MD,2017,5,TSN,96652.48,0.0,0.0
HI,2014,3,DPV,44233.17,0.0,0.0
NH,2017,1,TSN,4616.07,0.0,0.0


In [53]:
# list of NERC regions
NERCS = nerc_fraction.index.get_level_values('nerc').unique()

In [54]:
nercs

Index(['ASCC', 'SERC', 'SPP', 'WECC', 'NPCC', 'RFC', 'FRCC', 'HICC', 'MRO',
       'TRE'],
      dtype='object', name='nerc')

### Group the facility data to state fuel categories and add state labels

In [64]:
cols = list(nerc_frac_match.values())
eia_fac_type = group_fuel_cats(eia_fac, STATE_FACILITY_FUELS)
eia_fac_type = add_facility_location(eia_fac_type, location_labels, ['state', 'year'])
eia_fac_type = eia_fac_type.groupby(['state', 'year', 'month', 'type'])[cols].sum()

### Calculate the extra gen/fuel consumption at the state levels
**Only worrying about extra generation from 2016 forward.**

In [67]:
state_extra = (state_total.loc[idx[:, prev_year:, :, :], :]
               - eia_fac_type.loc[idx[:, prev_year:, :, :], :])
state_extra.dropna(how='all', inplace=True)
state_extra = state_extra.reorder_levels(['year', 'state', 'month', 'type'])
state_extra.sort_index(inplace=True)

Sort the index of each dataframe to make sure they can be easily combined.

In [65]:
nerc_fraction.sort_index(inplace=True)

In [68]:
state_extra.sort_index(inplace=True)

Create a copy of the `nerc_fraction` dataframe with repeated values for every month of the year, so that they MultiIndex matches the `state_extra` MultiIndex

In [69]:
df_list = []
for month in range(1, 13):
    df = nerc_fraction.copy()
    df['month'] = month
    df.set_index('month', append=True, inplace=True)
    df_list.append(df)

nerc_frac_monthly = pd.concat(df_list, axis=0)
nerc_frac_monthly.sort_index(inplace=True)
nerc_frac_monthly = (nerc_frac_monthly
                     .reorder_levels(['nerc', 'state', 'month', 'type']))

Cycle through each year (2016 and 2017 in this case) and each NERC, multiplying the state-level extra generation, total fuel consumption, and fuel consumption for electricity by the share that should be allocated to each NERC.

In [75]:
df_list_outer = []
for year in [FINAL_DATA_YEAR]:
    df_list_inner = []
    for nerc in nercs:
        df = pd.concat([(nerc_frac_monthly
                         .loc[nerc]['% generation']
                         * state_extra
                         .loc[year]['generation (mwh)']).dropna(),
                        (nerc_frac_monthly.
                         loc[nerc]['% total fuel']
                         * state_extra
                         .loc[year]['total fuel (mmbtu)']).dropna(),
                        (nerc_frac_monthly
                         .loc[nerc]['% elec fuel']
                         * state_extra
                         .loc[year]['elec fuel (mmbtu)']).dropna()],
                        axis=1)
        df.columns = nerc_frac_match.values()
        df['nerc'] = nerc
        df['year'] = year
        df = df.groupby(['year', 'nerc', 'month', 'type']).sum()
        df_list_inner.append(df)

    df_list_outer.append(pd.concat(df_list_inner))
final = pd.concat(df_list_outer)
final.sort_index(inplace=True)

In [76]:
nerc_frac_monthly.sort_index(inplace=True)

Although the dataframe is called `final`, it's really just the final allocated extra state-level generation/fuel consumption

In [79]:
# path = join(data_path, 'Derived data',
#             'NERC extra gen fuels {}.csv'.format(file_date))

out_path = (
    DATA_PATHS['nerc_extra']
    / f'NERC extra gen fuels {DATA_DATE}.csv'
)
final.to_csv(out_path)